In [6]:
import pennylane as qml
from pennylane import numpy as np
import math

dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def encode_voxel(x):
    # x scalar in [0,1]
    angle = 2.0 * math.asin(math.sqrt(float(x)))
    qml.RY(angle, wires=0)
    return qml.state()  # full state vector

# Testing with a few values
for x in [0.0, 0.25, 0.5, 0.75, 1.0]:
    state = encode_voxel(x)
    probs = np.abs(state)**2
    print(f"x={x: .2f} -> state={np.round(state,3)}, probs={np.round(probs,3)}")


x= 0.00 -> state=[1.+0.j 0.+0.j], probs=[1. 0.]
x= 0.25 -> state=[0.866+0.j 0.5  +0.j], probs=[0.75 0.25]
x= 0.50 -> state=[0.707+0.j 0.707+0.j], probs=[0.5 0.5]
x= 0.75 -> state=[0.5  +0.j 0.866+0.j], probs=[0.25 0.75]
x= 1.00 -> state=[0.+0.j 1.+0.j], probs=[0. 1.]


In [7]:
import pennylane as qml
from pennylane import numpy as np
import math

dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def encode_with_phase(x, theta0, theta1):
    # encode amplitude
    angle = 2.0 * math.asin(math.sqrt(float(x)))
    qml.RY(angle, wires=0)
    # mimic paper's two-phase param by applying RZ(theta0) then RZ(theta1)
    qml.RZ(float(theta0), wires=0)
    qml.RZ(float(theta1), wires=0)
    return qml.state()

# Trying some combos
states = [
    encode_with_phase(0.6, 0.0, 0.0),
    encode_with_phase(0.6, 0.5, 0.0),
    encode_with_phase(0.6, 0.0, 1.0),
]
for i, s in enumerate(states):
    print(f"case {i+1} state={np.round(s,3)} probs={np.round(np.abs(s)**2,3)}")


case 1 state=[0.632+0.j 0.775+0.j] probs=[0.4 0.6]
case 2 state=[0.613-0.156j 0.751+0.192j] probs=[0.4 0.6]
case 3 state=[0.555-0.303j 0.68 +0.371j] probs=[0.4 0.6]


In [5]:
import pennylane as qml
from pennylane import numpy as np
import math

dev = qml.device("default.qubit", wires=2)

# separate QNodes for state and probabilities
@qml.qnode(dev)
def two_voxel_crx_state(x0, x1, w01, w10):
    # encode voxel0 -> wire0, voxel1 -> wire1
    qml.RY(2*math.asin(math.sqrt(float(x0))), wires=0)
    qml.RY(2*math.asin(math.sqrt(float(x1))), wires=1)

    # applying phase (small)
    qml.RZ(0.1, wires=0)
    qml.RZ(0.2, wires=1)

    # controlled rotations:
    qml.CRX(float(w01)*float(x1), wires=[1,0])  # control=1 -> target=0 (voxel1 -> voxel0)
    qml.CRX(float(w10)*float(x0), wires=[0,1])  # control=0 -> target=1 (voxel0 -> voxel1)

    return qml.state()

@qml.qnode(dev)
def two_voxel_crx_probs(x0, x1, w01, w10):
    # encode voxel0 -> wire0, voxel1 -> wire1
    qml.RY(2*math.asin(math.sqrt(float(x0))), wires=0)
    qml.RY(2*math.asin(math.sqrt(float(x1))), wires=1)

    # applying phase (small)
    qml.RZ(0.1, wires=0)
    qml.RZ(0.2, wires=1)

    # controlled rotations:
    qml.CRX(float(w01)*float(x1), wires=[1,0])  # control=1 -> target=0 (voxel1 -> voxel0)
    qml.CRX(float(w10)*float(x0), wires=[0,1])  # control=0 -> target=1 (voxel0 -> voxel1)

    return qml.probs(wires=[0,1])

# test
state = two_voxel_crx_state(0.7, 0.3, w01=1.5, w10=0.5)
probs = two_voxel_crx_probs(0.7, 0.3, w01=1.5, w10=0.5)
print("Full state:", np.round(state,4))
print("Joint probs (|00|01|10|11):", np.round(probs,4))

Full state: [0.4531-0.0685j 0.3074-0.0865j 0.6884-0.1119j 0.4321-0.1218j]
Joint probs (|00|01|10|11): [0.21   0.1019 0.4865 0.2016]


In [8]:
import pennylane as qml
from pennylane import numpy as np
import itertools, math

N_QUBITS = 3
dev = qml.device("default.qubit", wires=N_QUBITS)

def tensor_pauliZ(indices):
    """Return Kronecker product of PauliZ on given wires"""
    mats = []
    for i in range(N_QUBITS):
        if i in indices:
            mats.append(np.array([[1, 0], [0, -1]]))  # PauliZ
        else:
            mats.append(np.eye(2))  # Identity
    M = mats[0]
    for m in mats[1:]:
        M = np.kron(M, m)
    return M

def measure_features(x_vals):
    features = []

    # ---  (singles, pairs, triples)
    all_sets = []
    all_sets += [[i] for i in range(N_QUBITS)]                          # singles
    all_sets += list(itertools.combinations(range(N_QUBITS), 2))       # pairs
    all_sets += list(itertools.combinations(range(N_QUBITS), 3))       # triples

    for inds in all_sets:
        mat = tensor_pauliZ(inds)

        @qml.qnode(dev)
        def circuit(x_vals):
            for j in range(N_QUBITS):
                angle = 2 * math.asin(math.sqrt(float(x_vals[j])))
                qml.RY(angle, wires=j)
            return qml.expval(qml.Hermitian(mat, wires=range(N_QUBITS)))

        features.append(circuit(x_vals))

    return features

# --- test
x = [0.6, 0.4, 0.9]
res = measure_features(x)

print("Features:", np.round(res, 3))
print("Total features =", len(res))



Features: [-0.2    0.2   -0.8   -0.04   0.16  -0.16   0.032]
Total features = 7


In [20]:
# step5_pytorch_wrapper.py
import torch
import torch.nn as nn
import torch.optim as optim
import pennylane as qml
import math
from pennylane import numpy as np

N_VOXELS = 4
dev = qml.device("default.qubit", wires=N_VOXELS)

# QNode returns list of per-qubit Z expectations (compact)
@qml.qnode(dev, interface="torch")
def qnode_expectations(x, phase_params, W_flat):
    # x: tensor shape (N_VOXELS,)
    for j in range(N_VOXELS):
        angle = 2.0 * math.asin(math.sqrt(float(x[j])))
        qml.RY(angle, wires=j)
    # phases
    for j in range(N_VOXELS):
        qml.RZ(float(phase_params[j]), wires=j)
    # pairwise CRX
    W = W_flat.reshape((N_VOXELS, N_VOXELS))
    for j in range(N_VOXELS):
        for k in range(N_VOXELS):
            if j==k: continue
            qml.CRX(float(W[j,k])*float(x[k]), wires=[k,j])
    # return local Z expectations as tensor
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(N_VOXELS)]

class QuantumBrainQubitTorch(nn.Module):
    def __init__(self, n_vox=N_VOXELS, out_dim=64):
        super().__init__()
        self.n = n_vox
        # learnable params
        self.phase = nn.Parameter(0.01*torch.randn(self.n))
        self.W = nn.Parameter(0.1*torch.randn(self.n * self.n))
        # small MLP to map q-features -> final features
        self.mlp = nn.Sequential(nn.Linear(self.n, 128), nn.ReLU(), nn.Linear(128, out_dim))

    def forward(self, x_batch):
        batch_size = x_batch.shape[0]
        q_feats = []
        for b in range(batch_size):
            x = x_batch[b]
            q_out = qnode_expectations(x, self.phase, self.W)
            q_feats.append(torch.tensor([float(v) for v in q_out], dtype=torch.float32))
        q_feats = torch.stack(q_feats, dim=0)
        return self.mlp(q_feats)

# toy training
def train_toy():
    torch.manual_seed(0)
    model = QuantumBrainQubitTorch()
    opt = optim.Adam(model.parameters(), lr=1e-3)
    # dummy data
    X = torch.rand((32, N_VOXELS))
    Y = torch.randn((32, 64))
    model.train()
    for epoch in range(6):
        perm = torch.randperm(X.size(0))
        loss_sum = 0.0
        for i in range(0, X.size(0), 8):
            idx = perm[i:i+8]
            xb = X[idx]
            yb = Y[idx]
            opt.zero_grad()
            out = model(xb)
            loss = nn.functional.mse_loss(out, yb)
            loss.backward()
            opt.step()
            loss_sum += float(loss.item()) * xb.size(0)
        print(f"Epoch {epoch+1}, loss: {loss_sum / X.size(0):.4f}")

if __name__ == "__main__":
    train_toy()


Epoch 1, loss: 1.0340
Epoch 2, loss: 1.0075
Epoch 3, loss: 0.9891
Epoch 4, loss: 0.9734
Epoch 5, loss: 0.9591
Epoch 6, loss: 0.9480


In [9]:
import math
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
import pennylane as qml

# -------------------------

# -------------------------
N_VOXELS = 4        # keep small for simulation (4 is okay); increase carefully
FEATURE_DIM = 64
LR = 2e-3
EPOCHS = 8
BATCH = 4

dev = qml.device("default.qubit", wires=N_VOXELS)

# -------------------------
# Defining a single-qubit unitary (RZ-RY-RZ) as a function
# We'll later wrap it with qml.ctrl to make it controlled by another wire.
# -------------------------
def single_qubit_unitary(a, b, c, wires=None):
    # a, b, c are rotation angles
    qml.RZ(a, wires=wires)
    qml.RY(b, wires=wires)
    qml.RZ(c, wires=wires)


# -------------------------
# QNode: parameterized circuit using learnable pairwise controlled unitaries
# We'll pass flattened parameters from PyTorch and reshape inside.
# Interface: "torch" so autograd works
# -------------------------
@qml.qnode(dev, interface="torch")
def qnode_learnable(x, phase_params, U_params_flat):
    """
    x: tensor shape (N_VOXELS,) values in [0,1]
    phase_params: tensor shape (N_VOXELS,)
    U_params_flat: flattened tensor containing 3 params per ordered pair (j,k), shape (N*N*3,)
    """
    # 1) Amplitude encode each voxel into a qubit
    for j in range(N_VOXELS):
        val = float(x[j])
        angle = 2.0 * math.asin(math.sqrt(max(0.0, min(1.0, val))))
        qml.RY(angle, wires=j)

    # 2) Phase shifts
    for j in range(N_VOXELS):
        qml.RZ(float(phase_params[j]), wires=j)

    # 3) Applying controlled U^{(j,k)} for each ordered pair (j,k), j != k
    # U_params_flat laid out as: for j in 0..N-1: for k in 0..N-1: [a,b,c] (diagonal entries can be zeros but we skip j==k)
    U_params = U_params_flat.reshape((N_VOXELS, N_VOXELS, 3))
    for j in range(N_VOXELS):
        for k in range(N_VOXELS):
            if j == k:
                continue
            a = float(U_params[j, k, 0])
            b = float(U_params[j, k, 1])
            c = float(U_params[j, k, 2])
            # Controlled single_qubit_unitary with control = k, target = j
            # Use qml.ctrl to create a controlled version of the function
            controlled_unitary = qml.ctrl(single_qubit_unitary, control=k)
            # Call it: pass angles and wires=target
            controlled_unitary(a, b, c, wires=j)

    # 4) Measurements: local Z expectations (compact features)
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(N_VOXELS)]


# -------------------------
# PyTorch Module wrapper
# -------------------------
class QuantumBrainLearnableUnitary(nn.Module):
    def __init__(self, n_vox=N_VOXELS, feature_dim=FEATURE_DIM):
        super().__init__()
        self.n = n_vox
        # learnable phase per qubit
        self.phase = nn.Parameter(0.01 * torch.randn(self.n))
        # learnable U params per ordered pair (j,k) -> 3 params per pair
        # shape: (n, n, 3)
        self.U_params = nn.Parameter(0.05 * torch.randn(self.n, self.n, 3))
        # classical projector from n quantum expvals -> feature_dim
        self.classical_proj = nn.Sequential(
            nn.Linear(self.n, 128),
            nn.ReLU(),
            nn.Linear(128, feature_dim)
        )

    def forward(self, x_batch):
        batch_size = x_batch.shape[0]
        q_feats = []
        for b in range(batch_size):
            x = x_batch[b]
            # Flatten U params to pass into qnode (qnode expects flat tensor)
            U_flat = self.U_params.flatten()
            q_out = qnode_learnable(x, self.phase, U_flat)
            # q_out is a list of torch scalars; convert to tensor
            q_tensor = torch.tensor([float(val) for val in q_out], dtype=torch.float32)
            q_feats.append(q_tensor)
        q_feats = torch.stack(q_feats, dim=0)  # (batch, n)
        return self.classical_proj(q_feats)


# -------------------------
# Toy training loop (MSE to random targets)
# -------------------------
def train_toy():
    torch.manual_seed(0)
    model = QuantumBrainLearnableUnitary()
    opt = optim.Adam(model.parameters(), lr=LR)

    # Toy dataset (random voxel activations and random "target" features)
    X = torch.rand((32, N_VOXELS))
    Y = torch.randn((32, FEATURE_DIM))

    model.train()
    for ep in range(EPOCHS):
        perm = torch.randperm(X.size(0))
        epoch_loss = 0.0
        for i in range(0, X.size(0), BATCH):
            idx = perm[i:i + BATCH]
            xb = X[idx]
            yb = Y[idx]
            opt.zero_grad()
            out = model(xb)
            loss = nn.functional.mse_loss(out, yb)
            loss.backward()
            opt.step()
            epoch_loss += float(loss.item()) * xb.size(0)
        epoch_loss /= X.size(0)
        print(f"Epoch {ep+1}/{EPOCHS}, loss: {epoch_loss:.6f}")

   
    model.eval()
    sample = torch.rand((1, N_VOXELS))
    with torch.no_grad():
        feat = model(sample)
    print("Sample feature norm:", torch.norm(feat).item())


if __name__ == "__main__":
    train_toy()


Epoch 1/8, loss: 1.054857
Epoch 2/8, loss: 0.989634
Epoch 3/8, loss: 0.960435
Epoch 4/8, loss: 0.937942
Epoch 5/8, loss: 0.919923
Epoch 6/8, loss: 0.902303
Epoch 7/8, loss: 0.885002
Epoch 8/8, loss: 0.870217
Sample feature norm: 2.190124034881592


In [3]:

import torch
import torch.nn as nn
import torch.optim as optim


class QuantumBrainQubits(nn.Module):
    def __init__(self):
        super(QuantumBrainQubits, self).__init__()
        # Define your model architecture here
        self.N_QUBITS = 4  # Assuming N_QUBITS is 4, adjust as needed
        self.layers = nn.Sequential(
            nn.Linear(self.N_QUBITS, 10),
            nn.ReLU(),
            nn.Linear(10, self.N_QUBITS)
        )
    
    def forward(self, x):
        return self.layers(x)


if __name__ == "__main__":
    N_QUBITS = 4  # Define N_QUBITS which was also missing
    
    model = QuantumBrainQubits()
    opt = optim.Adam(model.parameters(), lr=0.05)
    loss_fn = nn.MSELoss()

    X = torch.rand((40, N_QUBITS))
    y = X.sum(dim=1, keepdim=True)

    for epoch in range(1, 6):
        opt.zero_grad()
        out = model(X)
        pred = out.sum(dim=1, keepdim=True)
        loss = loss_fn(pred, y)
        loss.backward()
        opt.step()
        print(f"[QUBITS] Epoch {epoch}, Loss = {loss.item():.4f}")

[QUBITS] Epoch 1, Loss = 6.6837
[QUBITS] Epoch 2, Loss = 3.5966
[QUBITS] Epoch 3, Loss = 1.2550
[QUBITS] Epoch 4, Loss = 0.0502
[QUBITS] Epoch 5, Loss = 0.7128
